In [ ]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime
import warnings
warnings.filterwarnings('ignore')
from appscript import app, k
from mactypes import Alias
from pathlib import Path
from datetime import timedelta

In [ ]:
def generate_graph(df,variable,name_of_the_variable):
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Scatter(x=df['Time stamp'], y=df[variable], name=name_of_the_variable),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(x=df['Time stamp'], y=df['Threshold'], name="Threshold"),
        secondary_y=False,
    )


    fig.add_trace(
        go.Scatter(x=df['Time stamp'], y=df['Flag'], name="Flag",text=df["alert text"],textposition="bottom center"\
                   ,mode='lines+markers+text',marker=dict(color=df['color_marker']),line=dict(color='#00FE35')),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="<b>Alert Generation Graph</b>",title_x=0.5
    )

    # Set x-axis title
    # fig.update_xaxes(title_text="xaxis title")

    # Set y-axes titles
    fig.update_yaxes(title_text="<b>"+ name_of_the_variable +"</b> ", secondary_y=False)
    fig.update_yaxes(title_text="<b>Alert</b>", secondary_y=True)

    fig.show()
    return 

# Alerts Generation

In [ ]:
def generate_alert_report(df,variable,threshold,name_of_the_variable,case,obs_count):
    start=datetime.datetime.now()
#     print("Alert Report Generation Starts @ "+ str(start))

    df['Flag']=0
    if case=="greater":
        df.loc[df[variable]>threshold,'Flag']=1  
    else:
        df.loc[df[variable]<threshold,'Flag']=1  

    df['Alert Log']="No Alert"
    
    count=0
    add=[]
    z=0
    
    for i in range(obs_count,df['Flag'].shape[0]):
#         print(df['Time stamp'][i])
        j=i-obs_count
        while j< i:
            if df['Flag'][i]>=df['Flag'][j] & df['Flag'][i]!=0:
                count+=1
                if count>obs_count and ((df['Alert Log'][i-1]!="Alert") and (df['Alert Log'][i-2]!="Alert")):
                    z+=1
#                     print("Alert"+df['Time stamp'][i])
                    add.append(df['Time stamp'][i])
                    df['Alert Log'].iloc[i]="Alert"
    #                 print(z)
                    if z==1:
                        abc=1
#                         print("Email Sent")
    #                     create_message_with_attachment(df['Time stamp'][i])
                    break
                    count=6
            else:
                break
            j+=1


    add=list(set(add))
    len(add)

#     print("Total Number of Alerts:" + str(len(add)))

    df['color_marker']=np.where(df['Alert Log']=="Alert", 'red', '#00FE35')
    df['Threshold']=threshold

    df['alert text']=np.where(df['Alert Log']=="Alert","Alert","")
    
#     generate_graph(df,variable,name_of_the_variable)
    
#     print("Alert Report Generation Ends @ "+ str(datetime.datetime.now()))
#     print("************************************************************************************")
    return df

In [ ]:
def generate_alert_report(df,variable,threshold,name_of_the_variable,case,obs_count):
#     start=datetime.now()
#     print("Alert Report Generation Starts @ "+ str(start))
#     df['Time stamp']=df['Time stamp'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
    df=df.sort_values("Time stamp")
    df.reset_index(drop=True,inplace=True)
    df['Flag']=0
    if case=="greater":
        df.loc[df[variable]>threshold,'Flag']=1  
    else:
        df.loc[df[variable]<threshold,'Flag']=1  

    df['Alert Log']="No Alert"
    
    count=0
    add=[]
    z=0
    
    for i in range(obs_count,df['Flag'].shape[0]):
        count=0
        j=i-obs_count
        if df['Flag'][i]==1:
            while j<i:
                if (df['Flag'][j]==1):
                    count+=1
                    if count>obs_count-1 and ((df['Flag'][i-1]==1) and (df['Alert Log'][i-1]!="Alert")) \
                        and ((df['Alert Log'][i-2]!="Alert") and (df['Flag'][i-2]==1))\
                            and ((df['Alert Log'][i-3]!="Alert") and (df['Flag'][i-3]==1)):
                        z+=1
                        add.append(df['Time stamp'][i])
                        df['Alert Log'].iloc[i]="Alert"
                        if z==1:
                            abc=1
        #                     create_message_with_attachment(df['Time stamp'][i])
                    j+=1
    #                     break
    #                     count=obs_count
                else:
                    break
        else:
            pass
            


    add=list(set(add))
    len(add)

#     display(("## Total Number of Alerts:" + str(len(add))))

    df['color_marker']='#00FE35'
    df.loc[df['Alert Log']=="Alert",'color_marker']="red"
    df['Threshold']=threshold

    df['alert text']=np.where(df['Alert Log']=="Alert","Alert","")
    
#     generate_graph(df,variable,name_of_the_variable)
    
#     print("Alert Report Generation Ends @ "+ str(datetime.now()))
#     print("************************************************************************************************************************************)")
    return df

# Voltage Imbalance Alerts

In [ ]:
files=os.listdir("new_data2")

voltage_file=[]
for file in files:
    if file.find("-v")!=-1:
        voltage_file.append(file)

final_voltage=pd.DataFrame()
for file_ in voltage_file:
    temp=pd.read_csv("new_data2/"+file_)
    final_voltage=pd.concat([final_voltage,temp],axis=0)

final_voltage.drop_duplicates(inplace=True)
final_voltage.reset_index(drop=True,inplace=True)

final_voltage.sort_values("Time stamp",inplace=True)

In [ ]:
mains_voltage=final_voltage.columns[2:5]

final_voltage['Mean_Voltage']=(final_voltage[mains_voltage[0]]+final_voltage[mains_voltage[1]]+final_voltage[mains_voltage[2]])/3

final_alert=pd.DataFrame()
for vol in mains_voltage:
#     print(vol)
    final_voltage[vol+" %"]=round(abs(final_voltage[vol]-final_voltage['Mean_Voltage'])*100/final_voltage['Mean_Voltage'],2)
    variable=vol+" %"
    threshold=2
    name_of_the_variable="Voltage"
    case='greater'
    check=final_voltage[['Time stamp',vol,"Mean_Voltage",variable]]
    check.rename(columns={vol:"Voltage",variable:'Voltage %'},inplace=True)
    check['variable']=vol
    temp_=generate_alert_report(check,"Voltage %",threshold,name_of_the_variable,case,6)
    final_alert=pd.concat([final_alert,temp_],axis=0)

In [ ]:
# mains_voltage=final_voltage.columns[4:7]

# final_voltage['Mean_Voltage']=(final_voltage[mains_voltage[0]]+final_voltage[mains_voltage[1]]+final_voltage[mains_voltage[2]])/3

# # final_alert=pd.DataFrame()
# for vol in mains_voltage:
#     print(vol)
#     final_voltage[vol+" %"]=round(abs(final_voltage[vol]-final_voltage['Mean_Voltage'])*100/final_voltage['Mean_Voltage'],2)
#     variable=vol+" %"
#     threshold=2
#     name_of_the_variable="Voltage"
#     case='greater'
#     check=final_voltage[['Time stamp',vol,"Mean_Voltage",variable]]
#     check.rename(columns={vol:"Voltage",variable:'Voltage %'},inplace=True)
#     check['variable']=vol
#     temp_=generate_alert_report(check,"Voltage %",threshold,name_of_the_variable,case,6)
#     final_alert=pd.concat([final_alert,temp_],axis=0)

In [ ]:
final_voltage_alert=final_alert.copy()

# Phase Imbalance

In [ ]:
files=os.listdir("new_data2")

voltage_file=[]
for file in files:
    if file.find("-c")!=-1:
        voltage_file.append(file)

final_current=pd.DataFrame()
for file_ in voltage_file:
    temp=pd.read_csv("new_data2/"+file_)
    final_current=pd.concat([final_current,temp],axis=0)

final_current.drop_duplicates(inplace=True)
final_current.reset_index(drop=True,inplace=True)

final_current.sort_values("Time stamp",inplace=True)

In [ ]:
mains_voltage=final_current.columns[2:5]

final_current['mains_mean_Current']=(final_current[mains_voltage[0]]+final_current[mains_voltage[1]]+final_current[mains_voltage[2]])/3

final_alert=pd.DataFrame()
for vol in mains_voltage:
#     print(vol)
    final_current[vol+" %"]=round(abs(final_current[vol]-final_current['mains_mean_Current'])*100/final_current['mains_mean_Current'],2)
    variable=vol+" %"
    threshold=15
    name_of_the_variable="Current"
    case='greater'
    check=final_current[['Time stamp',vol,"mains_mean_Current",variable]]
    check.rename(columns={vol:"Current",variable:'Current %'},inplace=True)
    check['variable']=vol
    temp_=generate_alert_report(check,"Current %",threshold,name_of_the_variable,case,6)
    final_alert=pd.concat([final_alert,temp_],axis=0)

In [ ]:
# mains_voltage=final_current.columns[4:7]

# final_current['mains_mean_Current']=(final_current[mains_voltage[0]]+final_current[mains_voltage[1]]+final_current[mains_voltage[2]])/3

# # final_alert=pd.DataFrame()
# for vol in mains_voltage:
#     print(vol)
#     final_current[vol+" %"]=round(abs(final_current[vol]-final_current['mains_mean_Current'])*100/final_current['mains_mean_Current'],2)
#     variablea=vol+" %"
#     threshold=15
#     name_of_the_variable="Current"
#     case='greater'
#     check=final_current[['Time stamp',vol,"mains_mean_Current",variable]]
#     check.rename(columns={vol:"Current",variable:'Current %'},inplace=True)
#     check['variable']=vol
#     temp_=generate_alert_report(check,"Current %",threshold,name_of_the_variable,case,6)
#     final_alert=pd.concat([final_alert,temp_],axis=0)

In [ ]:
final_current_alert=final_alert.copy()

In [ ]:
# mains_voltage=['Main 1 L1   (A)', 'Main 1 L2   (A)','Main 1 L3   (A)']

# df['mains_mean_Current']=(df[mains_voltage[0]]+df[mains_voltage[1]]+df[mains_voltage[2]])/3

# for vol in mains_voltage:
#     print(vol)
#     df[vol+" %"]=round(abs(df[vol]-df['mains_mean_Current'])*100/df['mains_mean_Current'],2)
#     variable=vol+" %"
#     threshold=15
#     name_of_the_variable="Current"
#     case='greater'
#     generate_alert_report(variable,threshold,name_of_the_variable,case)

# Power Factor Alerts

In [ ]:
files=os.listdir("new_data2")

voltage_file=[]
for file in files:
    if file.find("-p")!=-1:
        voltage_file.append(file)

final_pf=pd.DataFrame()
for file_ in voltage_file:
    temp=pd.read_csv("new_data2/"+file_)
    final_pf=pd.concat([final_pf,temp],axis=0)

final_pf.drop_duplicates(inplace=True)
final_pf.reset_index(drop=True,inplace=True)

# final_pf.sort_values("Time stamp",inplace=True)

In [ ]:
power_factor=final_pf.columns[1:]
final_alert=pd.DataFrame()
for pf in power_factor:
#     print(pf)
    variable="PF"
    threshold=0.9
    name_of_the_variable="Power Factor"
    case='lesser'
    check=final_pf[['Time stamp',pf]]
    check.rename(columns={pf:"PF"},inplace=True)
    check['variable']=pf
    temp_=generate_alert_report(check,variable,threshold,name_of_the_variable,case,6)
    final_alert=pd.concat([final_alert,temp_],axis=0)

In [ ]:
final_pf_alert=final_alert.copy()

In [ ]:
# power_factor=final_pd.columns[1:]
# for pf in power_factor:
#     print(pf)
#     variabale=pf
#     threshold=0.9
#     name_of_the_variable="Power Factor"
#     case='lesser'
#     generate_alert_report(variable,threshold,name_of_the_variable,case)

In [ ]:
files=os.listdir("new_data2")

voltage_file=[]
for file in files:
    if file.find("-e")!=-1:
        voltage_file.append(file)

energy=pd.DataFrame()
for file_ in voltage_file:
    temp=pd.read_csv("new_data2/"+file_)
    energy=pd.concat([energy,temp],axis=0)

energy.drop_duplicates(inplace=True)
energy.reset_index(drop=True,inplace=True)

energy.sort_values("Time stamp",inplace=True)

In [ ]:
files=os.listdir("new_data2/benchmark")

voltage_file=[]
for file in files:
    if file.find("-e")!=-1:
        voltage_file.append(file)

energy1=pd.DataFrame()
for file_ in voltage_file:
    temp=pd.read_csv("new_data2/benchmark/"+file_)
    energy1=pd.concat([energy1,temp],axis=0)

energy1.drop_duplicates(inplace=True)
energy1.reset_index(drop=True,inplace=True)

energy1.sort_values("Time stamp",inplace=True)

In [ ]:
energy=pd.concat([energy,energy1],axis=0)

In [ ]:
energy=energy.sort_values("Time stamp")

In [ ]:
df=energy.copy()
df['Time stamp']=df['Time stamp'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M'))
df = df.sort_values('Time stamp')
df['hour']=df['Time stamp'].dt.hour
df['weekday'] = df['Time stamp'].dt.day_name()
df['trading_cat'] = df['hour'].apply(lambda x: 'trading' if (x >= 14 and x <= 23) else ('non-trading' if (x >= 0 and x <= 10) else 'sleep'))
df.loc[(df['weekday'] == 'Monday') & (df['hour'] >10) & (df['hour'] <17), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Tuesday') & (df['hour'] >11) & (df['hour'] <17), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Wednesday') & (df['hour'] >14) & (df['hour'] <17), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Wednesday') & (df['hour'] >0) & (df['hour'] <15), 'trading_cat'] = 'non-trading'
df.loc[(df['weekday'] == 'Thursday') & (df['hour'] >14) & (df['hour'] <17), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Thursday') & (df['hour'] >0) & (df['hour'] <15), 'trading_cat'] = 'non-trading'
df.loc[(df['weekday'] == 'Friday') & (df['hour'] >13) & (df['hour'] <15), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Friday') & (df['hour'] >0) & (df['hour'] <14), 'trading_cat'] = 'non-trading'
df.loc[(df['weekday'] == 'Saturday') & (df['hour'] >12) & (df['hour'] <15), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Saturday') & (df['hour'] >0) & (df['hour'] <13), 'trading_cat'] = 'non-trading'
df.loc[(df['weekday'] == 'Sunday') & (df['hour'] >13) & (df['hour'] <15), 'trading_cat'] = 'sleep'
df.loc[(df['weekday'] == 'Sunday') & (df['hour'] >0) & (df['hour'] <14), 'trading_cat'] = 'non-trading'

In [ ]:
def generate_alert_report_energy(temp,variable,threshold,name_of_the_variable,case,obs_count):
#     start=datetime.now()
#     print("Alert Report Generation Starts @ "+ str(start))
#     temp['Time stamp']=temp['Time stamp'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
    temp=temp.sort_values("Time stamp")
    temp.reset_index(drop=True,inplace=True)
    temp['Flag']=0
    if case=="greater":
        temp.loc[temp[variable]>temp['threshold'],'Flag']=1  
    else:
        temp.loc[temp[variable]<temp['threshold'],'Flag']=1  

    temp['Alert Log']="No Alert"
    
    count=0
    add=[]
    z=0
    
    for i in range(obs_count,temp['Flag'].shape[0]):
#         print(temp['Time stamp'][i])
        count=0
        j=i-obs_count
        if temp['Flag'][i]==1:
            while j<i:
#                 print(j)
                if (temp['Flag'][j]==1):
                    count+=1
#                     print("No of count:"+str(count))
                    if count>obs_count-1 and ((temp['Flag'][i-1]==1) and (temp['Alert Log'][i-1]!="Alert")):#((temp['Alert Log'][i-1]!="Alert") and (temp['Alert Log'][i-2]!="Alert")):
                        z+=1
#                         print("Alert"+str(temp['Time stamp'][i]))
                        add.append(temp['Time stamp'][i])
                        temp['Alert Log'].iloc[i]="Alert"
        #                 print(z)
                        if z==1:
                            abc=1
    #                         print("Email Sent")
        #                     create_message_with_attachment(temp['Time stamp'][i])
                    j+=1
    #                     break
    #                     count=obs_count
                else:
                    break
        else:
            pass
            


    add=list(set(add))
    len(add)

#     print("Total Number of Alerts:" + str(len(add)))

#     temp['color_marker']=np.where(temp['Alert Log']=="Alert", 'red', '#00FE35')
    temp['color_marker']='#00FE35'
    temp.loc[temp['Alert Log']=="Alert",'color_marker']="red"
    temp['Threshold']=temp['threshold']

    temp['alert text']=np.where(temp['Alert Log']=="Alert","Alert","")
    
#     generate_graph(temp,variable,name_of_the_variable)
    
#     print("Alert Report Generation Ends @ "+ str(datetime.now()))
#     print("************************************************************************************")
    return temp

In [ ]:
def get_threshold(date_range,col,trad_cat):
    std=df[(df['Time stamp']<date_range.replace(hour=0)) &
           (df['Time stamp']>date_range-timedelta(days=30)) &
           (df['weekday']==date_range.day_name()) & 
           (df['trading_cat']==trad_cat)][col].std()
    mean=df[(df['Time stamp']<date_range.replace(hour=0)) & 
            (df['Time stamp']>date_range-timedelta(days=30)) &
            (df['weekday']==date_range.day_name()) &
            (df['trading_cat']==trad_cat)][col].mean()
    threshold=mean+std*2
    return threshold

In [ ]:
# flt_df=df[df['Time stamp']>="2021-07-01"]
flt_df=df[df['Time stamp']>="2021-07-31"]
final_energy_alert=pd.DataFrame()
for col in flt_df.columns[1:-3]:
    temp=flt_df[['Time stamp',col,'weekday','trading_cat']]
#     temp['threshold']=temp['Time stamp'].apply(lambda x: get_threshold(x,col))
    temp['threshold']=temp.apply(lambda x: get_threshold(x['Time stamp'],col,x['trading_cat']),axis=1)
    temp.rename(columns={col:"Energy"},inplace=True)
    temp['variable']=col
    case='greater'
    variable="Energy"
    name_of_the_variable="Energy"
    temp=temp.sort_values("Time stamp",ascending=0)
    temp_=generate_alert_report_energy(temp,variable,threshold,name_of_the_variable,case,2)
    final_energy_alert=pd.concat([final_energy_alert,temp_],axis=0)

In [ ]:
final_energy_alert[final_energy_alert['Alert Log']=="Alert"].to_csv("Energy_Alert.csv")

In [ ]:
current=final_current_alert[final_current_alert['Alert Log']=="Alert"]

In [ ]:
voltage=final_voltage_alert[final_voltage_alert['Alert Log']=="Alert"]

In [ ]:
pf=final_pf_alert[final_pf_alert['Alert Log']=="Alert"]

In [ ]:
energy=final_energy_alert[final_energy_alert['Alert Log']=="Alert"]

In [ ]:
current=current.groupby('variable')['Current'].count().to_frame("#Alerts").reset_index()
current['Alert Type']="Load Imbalance"

In [ ]:
voltage=voltage.groupby('variable')['Voltage'].count().to_frame("#Alerts").reset_index()
voltage['Alert Type']="Voltage Imbalance"

In [ ]:
pf=pf.groupby('variable')['PF'].count().to_frame("#Alerts").reset_index()
pf['Alert Type']="Low Power Factor"

In [ ]:
energy=energy.groupby('variable')['Energy'].count().to_frame("#Alerts").reset_index()
energy['Alert Type']="Abnormal Energy Consumption"

In [ ]:
final_alert_summary=pd.concat([current,voltage,pf,energy])

In [ ]:
final_alert_summary

In [ ]:
final_alert_summary['variable']=final_alert_summary['variable'].apply(lambda x: x.split("(")[0])

In [ ]:
final_alert_summary=pd.pivot_table(final_alert_summary,index="variable",columns="Alert Type",values="#Alerts").fillna(0)

In [ ]:
# final_alert_summary